In [1]:
%run -i 'setup.py'

import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from src import country_data
from src import ml_model

In [2]:
df = country_data.load_combined()
df

code  defence_budget  economic_affairs_budget  \
country       year                                                 
Australia     2000  AUS          810.00                  2305.06   
              2001  AUS          876.34                  2520.57   
              2002  AUS          952.50                  2471.85   
              2003  AUS          941.96                  2630.58   
              2004  AUS          954.41                  2729.79   
...                 ...             ...                      ...   
United States 2014  USA        60919.16                 59642.97   
              2015  USA        59874.15                 58900.14   
              2016  USA        59902.90                 62828.36   
              2017  USA        61595.47                 64257.30   
              2018  USA        64942.93                 67749.42   

                    education_budget  environment_protection_budget  \
country       year                                                    
Australia     2000           2703.66                         278.31   
              2001           2873.72                         317.33   
              2002           3054.12                         359.97   
              2003           3197.41                         363.18   
              2004           3420.64                         419.68   
...                              ...                            ...   
United States 2014         107557.50                           0.00   
              2015         111188.11                           0.00   
              2016         114157.80                           0.00   
              2017         117005.45                           0.00   
              2018         123364.21                           0.00   

                    general_public_services_budget  health_budget  \
country       year                                                  
Australia     2000                         2594.59        3054.12   
              2001                         2384.78        3321.49   
              2002                         2434.51        3519.82   
              2003                         2443.23        3783.56   
              2004                         2607.13        4202.01   
...                                            ...            ...   
United States 2014                       101519.01      155813.85   
              2015                       101614.44      166514.54   
              2016                       105975.28      174124.38   
              2017                       109919.14      181783.57   
              2018                       115892.79      191662.75   

                    housing_and_community_amenities_budget  \
country       year                                           
Australia     2000                                  420.02   
              2001                                  487.21   
              2002                                  463.23   
              2003                                  437.90   
              2004                                  502.36   
...                                                    ...   
United States 2014                                 9269.78   
              2015                                 9473.39   
              2016                                 9745.64   
              2017                                11629.72   
              2018                                12261.75   

                    public_order_and_safety_budget  \
country       year                                   
Australia     2000                          828.84   
              2001                          904.87   
              2002                          978.01   
              2003                         1040.06   
              2004                         1085.05   
...                                            ...   
United States 2014                        35493.18   
              2015 

## Improve data imputation?

In [25]:
def prepare_data(df):
    prepared_df = df.reset_index().drop("code", axis=1)

    features = prepared_df[ml_model.FEATURES]
    labels = prepared_df[ml_model.LABELS]

    return features, labels


current_model = ml_model.pipeline()

features, labels = prepare_data(df)

In [26]:
imputed_labels = labels.fillna(0)

X_train, X_test, y_train, y_test = ml_model.split_data(features, imputed_labels)

current_model.fit(X_train, y_train)
y_pred = current_model.predict(X_test)

mean_absolute_error(y_test, y_pred)

13.099286898202202

In [28]:
imputer = IterativeImputer(max_iter=100)

X_train, X_test, y_train, y_test = ml_model.split_data(features, labels)

imputer.fit(y_train)

current_model.fit(X_train, imputer.transform(y_train))
y_pred = current_model.predict(X_test)

mean_absolute_error(imputer.transform(y_test), y_pred)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


3.5120566675688294

### Iterative imputation is a little better than filling with zeros

They're made-up data labels, but they should give us more variety than making all the NaNs 0, and we don't have time to go find the actual data from other sources. In particular the ranges of values is a bit more realistic, because OECD countries probably don't have safe drinking water for only 36.4% of their residents on average.

## Add World Bank data

In [15]:
gini = os.path.join(BASE_DIR, 'data/country_stats/world_bank/gini_index/API_SI.POV.GINI_DS2_en_csv_v2_247786.csv')
wb_id_vars = ['Country Name', 'Country Code', 'Indicator Name']

gini_df = (
    pd
    .read_csv(gini, header=2)
    .drop('Indicator Code', axis=1)
    .melt(id_vars=wb_id_vars, value_vars=np.arange(1960, 2019).astype(str))
    .rename(columns={'Country Name': 'country', 'variable': 'year', 'value': 'gini_index'})
    .astype({'year': int})
    .set_index(['country', 'year'])
    .loc[:, ['gini_index']]
    .groupby('country')
    .ffill()
    .groupby('country')
    .bfill()
)


gini_df

,,gini_index
country,year,
Aruba,1960,NaN
Afghanistan,1960,NaN
Angola,1960,52.0
Albania,1960,27.0
Andorra,1960,NaN
...,...,...
Kosovo,2018,29.0
"Yemen, Rep.",2018,36.7
South Africa,2018,63.0


In [16]:
pop = os.path.join(BASE_DIR, 'data/country_stats/world_bank/population/API_SP.POP.TOTL_DS2_en_csv_v2_247892.csv')

pop_df = (
    pd
    .read_csv(pop, header=2)
    .drop('Indicator Code', axis=1)
    .melt(id_vars=wb_id_vars, value_vars=np.arange(1960, 2019).astype(str))
    .rename(columns={'Country Name': 'country', 'variable': 'year', 'value': 'population'})
    .astype({'year': int})
    .set_index(['country', 'year'])
    .loc[:, ['population']]
    .groupby('country')
    .ffill()
    .groupby('country')
    .bfill()
)


pop_df

,,population
country,year,
Aruba,1960,54211.0
Afghanistan,1960,8996973.0
Angola,1960,5454933.0
Albania,1960,1608800.0
Andorra,1960,13411.0
...,...,...
Kosovo,2018,1845300.0
"Yemen, Rep.",2018,28498687.0
South Africa,2018,57779622.0


In [18]:
expanded_df = df.join([gini_df, pop_df], how='left')
expanded_df

code  defence_budget  economic_affairs_budget  \
country       year                                                 
Australia     2000  AUS          810.00                  2305.06   
              2001  AUS          876.34                  2520.57   
              2002  AUS          952.50                  2471.85   
              2003  AUS          941.96                  2630.58   
              2004  AUS          954.41                  2729.79   
...                 ...             ...                      ...   
United States 2014  USA        60919.16                 59642.97   
              2015  USA        59874.15                 58900.14   
              2016  USA        59902.90                 62828.36   
              2017  USA        61595.47                 64257.30   
              2018  USA        64942.93                 67749.42   

                    education_budget  environment_protection_budget  \
country       year                                                    
Australia     2000           2703.66                         278.31   
              2001           2873.72                         317.33   
              2002           3054.12                         359.97   
              2003           3197.41                         363.18   
              2004           3420.64                         419.68   
...                              ...                            ...   
United States 2014         107557.50                           0.00   
              2015         111188.11                           0.00   
              2016         114157.80                           0.00   
              2017         117005.45                           0.00   
              2018         123364.21                           0.00   

                    general_public_services_budget  health_budget  \
country       year                                                  
Australia     2000                         2594.59        3054.12   
              2001                         2384.78        3321.49   
              2002                         2434.51        3519.82   
              2003                         2443.23        3783.56   
              2004                         2607.13        4202.01   
...                                            ...            ...   
United States 2014                       101519.01      155813.85   
              2015                       101614.44      166514.54   
              2016                       105975.28      174124.38   
              2017                       109919.14      181783.57   
              2018                       115892.79      191662.75   

                    housing_and_community_amenities_budget  \
country       year                                           
Australia     2000                                  420.02   
              2001                                  487.21   
              2002                                  463.23   
              2003                                  437.90   
              2004                                  502.36   
...                                                    ...   
United States 2014                                 9269.78   
              2015                                 9473.39   
              2016                                 9745.64   
              2017                                11629.72   
              2018                                12261.75   

                    public_order_and_safety_budget  \
country       year                                   
Australia     2000                          828.84   
              2001                          904.87   
              2002                          978.01   
              2003                         1040.06   
              2004                         1085.05   
...                                            ...   
United States 2014                        35493.18   
              2015 

In [24]:
def prepare_data(df):
    prepared_df = df.reset_index().drop("code", axis=1)

    features = prepared_df[ml_model.FEATURES + ['gini_index', 'population']]
    labels = prepared_df[ml_model.LABELS]

    return features, labels


features, labels = prepare_data(expanded_df)

X_train, X_test, y_train, y_test = ml_model.split_data(features, labels)

imputer.fit(y_train)

current_model.fit(X_train, imputer.transform(y_train))
y_pred = current_model.predict(X_test)

mean_absolute_error(imputer.transform(y_test), y_pred)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


3.452752410529173

## Try other algorithms

In [36]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [50]:
base_pipeline_steps = [
    ColumnTransformer(
        [
            (
                "onehotencoder",
                OneHotEncoder(sparse=False, handle_unknown="ignore"),
                ["country"],
            )
        ],
        remainder=StandardScaler(),
    )
]

regressors = [
    RandomForestRegressor(n_estimators=100),
    ExtraTreesRegressor(n_estimators=100),
    Lasso(),
    Ridge(),
    LinearRegression()
]

for regressor in regressors:
    pipeline_steps = base_pipeline_steps + [regressor]
    pipeline = make_pipeline(*pipeline_steps)

    pipeline.fit(X_train, imputer.transform(y_train))
    y_pred = pipeline.predict(X_test)

    print(regressor.__class__.__name__)
    print(mean_absolute_error(imputer.transform(y_test), y_pred))
    print()

RandomForestRegressor
0.6465553878303332

ExtraTreesRegressor
0.1781782894218609

Lasso
3.668246813640717

Ridge
1.2889855146030529

LinearRegression
1.24439617402233



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


### ExtraTrees has the best MAE

I tend to get better performance from gradient-boosted trees, but they don't support multi-output by default, so sticking with the best of the algorithms that doesn't require extra work.